### Imports

In [399]:
import numpy as np
import sympy as sym
from numpy.random import randn
np.set_printoptions(threshold=np.inf)


### Values

In [400]:
m = 5 # number of points that we want to use
sigma = 2
sigma2 = 2*sigma**2
w = randn(m) / sigma
b = np.random.rand(m)*2*np.pi

n = 5 # so many points that we have (Gauss uses all n points)
x = randn(n)/2
z = randn(n)/2 # function y

lam = 0.000001

d = 5

x1 = randn(n)/2
x2 =  randn(n)/2
x_vector = np.block([x1,x2])
print("x: ", x_vector)

y1 = np.cos(x1)
y2 = np.sin(x2)

y = np.block([y1, y2])
print("y: ", y)

x:  [ 0.43716881 -0.84503379  0.20729834  0.08254591  0.25320426 -0.20729648
 -0.24219505 -0.19546464  0.29174124 -0.08507908]
y:  [ 0.90595395  0.66370601  0.97859053  0.99659502  0.9681147  -0.20581501
 -0.23983419 -0.19422234  0.28762033 -0.08497647]


### Gaussian Kernel

In [401]:
def k_gauss(x,z):
    """R^n x R^n -> R - should be one number
    # vector is multi-dimensional

    Args:
        x (float): real value numbers
        z (float): real value numbers

    Returns:
        (vector with length n): a scalar shift-invariant Gaussian kernel
    """
    return np.exp(-((np.linalg.norm(x-z))**2)/(sigma2)) 

print(k_gauss(x,z))

0.9340607761199916


###  Scalar kernel

In [402]:
def k_approx(k, x, z):
    """scalar kernel

    Args:
        k (kernel): to test with different kernels

    Returns:
        approximated kernel: sum of feature vectors
    """
    n = len(x)
    Kxz = np.zeros((n,m))
    for i in range(n):
        for j in range(m):
            Kxz[i, j] = k(x[i], z[j])
    return Kxz

### Approximation 1

In [403]:
def psi_approx1(x, w):
   """
   Args:
       x (vector): vector with length m

   Returns:
       vector: vector with length m
   """
   return np.sqrt(2)*np.cos(w.T*x + b)

"""
def psi(x, d, list_w):
    Args:
        list_w (list): list of size d of all w, every w is of size n

    Returns:
        list: a list of cos and sin from 1 to 2d
    new_list = []
    for w in list_w:
        new_list.append(phi_cos(x,w))
        new_list.append(phi_sin(x,w))
    return (1/np.sqrt(d)) * np.array(new_list)

list_w = list_w_of_size_d(d)
print(list_w)
print("psi list:", psi(x, d, list_w))
"""

def psi_approx1_w(x, list_w):
    new_list = []
    for w in list_w:
        new_list.append(psi_approx1(x, w))
    return np.array(new_list)

print(psi_approx1_w(x, w))


[[ 0.58317932 -1.04924705  0.32836584  0.25770115 -0.50400597]
 [ 0.53101953 -1.02537059  0.49098714  0.32919502 -0.49636154]
 [ 0.19982447 -0.8677342   1.24930289  0.73688108 -0.44969967]
 [ 0.27043204 -0.90230607  1.13479112  0.65665024 -0.45946464]
 [ 0.51703933 -1.01893156  0.53302079  0.34801615 -0.49433138]]


### Approximation 2

In [404]:
def phi_cos(x, w):
    return np.cos(w.T@x)

def phi_sin(x, w):
    return np.sin(w.T@x)

def K_approx2(x, z, w):
    """The scalar kernel

    Args:
        x (float): random generated numbers
        z (float): random generated numbers

    Returns:
        (float): a number, which is the scalar kernel
    """
    return np.sum(phi_cos(x, w) * phi_cos(z, w)) + np.sum(phi_sin(x, w) * phi_sin(z, w))

"""
print(K_approx2(x,z))
"""

'\nprint(K_approx2(x,z))\n'

### Matrix kernels

In [405]:
# Matrix kernel
def K(k, *args): # *args: can give as many arguments as you want
    return k(*args) * np.eye(m)

# Testing for gaussian kernel
gauss_sep_ker = K(k_gauss, x, z)
print(gauss_sep_ker)

# Testing for approximation with RFF (random fourier features)
print("\n", "matrix kernel")
matrix_ker = K(K_approx2, x, z, w)
print(matrix_ker)

[[0.93406078 0.         0.         0.         0.        ]
 [0.         0.93406078 0.         0.         0.        ]
 [0.         0.         0.93406078 0.         0.        ]
 [0.         0.         0.         0.93406078 0.        ]
 [0.         0.         0.         0.         0.93406078]]

 matrix kernel
[[0.97157872 0.         0.         0.         0.        ]
 [0.         0.97157872 0.         0.         0.        ]
 [0.         0.         0.97157872 0.         0.        ]
 [0.         0.         0.         0.97157872 0.        ]
 [0.         0.         0.         0.         0.97157872]]


### Approximate function f

In [406]:
# create a list of all the w_1...w_d
def list_w_of_size_d(d):
    list_w = []
    for i in range(d):
        w = np.random.normal(size = d) # should be N or d times
        list_w.append(w)
    return np.array(list_w)

# Approximation 2
def psi(x, d, list_w):
    """
    Args:
        list_w (list): list of size d of all w, every w is of size n

    Returns:
        list: a list of cos and sin from 1 to 2d
    """
    new_list = []
    for w in list_w:
        new_list.append(phi_cos(x,w))
        new_list.append(phi_sin(x,w))
    return (1/np.sqrt(d)) * np.array(new_list)

list_w = list_w_of_size_d(d)
print(list_w)
print("psi list:", psi(x, d, list_w))

[[ 1.1816333   0.51850647 -1.13127822  0.91401341  1.15905029]
 [-0.13501085  0.3778589   0.42745275  1.20505983 -0.82969323]
 [ 0.45673803 -0.99239495  0.34751278 -0.3999147  -2.17369366]
 [-0.32412     0.808393   -0.32520323 -1.22286439  0.29158551]
 [-1.48006964  0.67491242  0.92852561 -0.15152077  0.19559318]]
psi list: [ 0.43886316  0.08601818  0.24052304 -0.37702609  0.44712592 -0.00885514
  0.3013671   0.33042075  0.38038119 -0.23518109]


### Phi

In [407]:
def phi(x, m, d, list_w):
    psi_vector = []
    j = 0
    for i in range(0, len(x)):
        psi_vector.append(psi_approx1(x[i], list_w))
    return np.array(psi_vector)

list_w = np.random.normal(size = d)
phi_ = phi(x_vector, m, d, list_w)
print("phi: ", phi_)

phi:  [[ 0.26530097 -0.48707915  1.23266386  0.45863313 -0.82322117]
 [ 0.26076633 -1.37704524  0.94074037  1.01053521  0.32585903]
 [ 0.26448821 -0.71370677  1.18941813  0.56860026 -0.63826305]
 [ 0.26404709 -0.82775143  1.16418798  0.62666976 -0.53134522]
 [ 0.26465053 -0.67001969  1.1983936   0.54693189 -0.67653646]
 [ 0.2630221  -1.06145865  1.10096276  0.7564204  -0.27009212]
 [ 0.26289868 -1.08623641  1.09292785  0.77150563 -0.23776207]
 [ 0.26306395 -1.05288249  1.10366666  0.75127827 -0.28102091]
 [ 0.26478679 -0.632694    1.20579861  0.52862413 -0.70817951]
 [ 0.26345432 -0.96874036  1.12839408  0.70264882 -0.38205049]]


### Alpha

In [408]:
def alpha(phi, y, m, lam):
    """
    input:
        phi - 10x5 matrix, 10 is length mN, 5 is length d
        y - [cos(x_vec) sin(x_vec)] dim: m*N

    return:
        alpha: we want alpha to be a matrix of len d
    """
    N = len(y)/m
    alpha = np.linalg.inv((np.transpose(phi) @ phi + lam * N * np.identity(m))) @ (np.transpose(phi) @ y)
    return alpha

alpha_ = alpha(phi_, y, m, lam)
print(alpha_)

[ -4.76538916  33.73572826 -85.19524056 148.79864069 -67.13997651]


### Optimization problem

In [409]:
def loss_function(y, alpha, phi, lam):
    N = len(y)/m
    y_matrix = np.transpose(y)@y
    phi_y = 2*(np.transpose(alpha)@np.transpose(phi)@y)
    phi_phi = np.transpose(alpha)@np.transpose(phi)@phi@alpha
    alpha_alpha = lam*(np.transpose(alpha)@alpha)
    loss = 1/N *(y_matrix-phi_y+phi_phi)+alpha_alpha
    return loss

loss_func = loss_function(y, alpha_, phi_, lam)
print(loss_func)

0.3596904028005414


### Function

In [410]:
def func(phi, alpha):
    return phi@alpha

def psi_vector(x, list_w):
    psi = []
    for i in range(len(x)):
        psi.append(psi_approx1(x[i], list_w))
    return psi

psi_ = psi_vector(x_vector, list_w)  

vector_F = func(phi_, alpha_)

print(vector_F)

[ 0.80170978  0.64321925  0.78934128  0.55575525  0.84316849 -0.17096388
 -0.24771282 -0.14358312  0.87120065  0.13362138]
